## Mean Reversion for a Singular Index - Testing Notebook

We will create a program that performs mean reversion on the SPY stock. The main purpose of this program is to build familiarity for how the different libraries interact and how backtesting can be done on a optimized solution.

This will be done in three steps:
1. import data using openbb
2. build sk-learn model to optimize mean reversion strategy
3. walk-forward backtesting on historical data using vectorbt

Afterward, we will perform a fourth step in order to assess the effectiveness of our strategy:
4. validation of results (through a simulation on data outside the overall training period)

In other programs, we will add a fourth step (train sk-learn model with inputs from ta-lib indicators), but the ta-lib indicators are not needed for the mean reversion strategy.

### Import Libraries

In [172]:
from openbb_terminal.sdk import openbb
import talib
import vectorbt as vbt
import numpy as np
import pandas as pd
import sklearn

### Import Data Using OpenBB

In [173]:
# grab data

ohlcv = openbb.stocks.load(symbol="SPY", start_date="2016-03-01", end_date="2023-01-30")
print(ohlcv)
print(type(ohlcv))

INFO:openbb_terminal.stocks.stocks_helper:START
INFO:openbb_terminal.stocks.stocks_helper:{"INPUT": {"start_date": "2016-03-01", "interval": "1440", "end_date": "2023-01-30", "prepost": "False", "source": "YahooFinance", "weekly": "False", "monthly": "False", "verbose": "True", "symbol": "SPY", "chart": "False"}, "VIRTUAL_PATH": "stocks.load", "CHART": false}


Loading Daily data for SPY with starting period 2016-03-01.

INFO:openbb_terminal.stocks.stocks_helper:END


                  Open        High         Low       Close   Adj Close  \
date                                                                     
2016-03-01  170.051015  172.841464  169.562691  172.754257  172.754257   
2016-03-02  172.431620  173.582668  172.004329  173.530350  173.530350   
2016-03-03  173.347214  174.227954  172.754252  174.210510  174.210510   
2016-03-04  174.411055  175.579560  173.556487  174.777298  174.777298   
2016-03-07  173.826810  175.335400  173.748332  174.916824  174.916824   
...                ...         ...         ...         ...         ...   
2023-01-24  394.501307  396.746377  393.274928  395.806824  395.806824   
2023-01-25  391.603507  396.301364  389.239728  395.955200  395.955200   
2023-01-26  398.704703  400.475062  395.638727  400.306915  400.306915   
2023-01-27  399.228870  403.679472  399.011284  401.226685  401.226685   
2023-01-30  398.378267  400.682706  395.885941  396.192535  396.192535   

               Volume  Dividends  Sto

In [174]:
# instantiate 200 day mean with close prices

closes = ohlcv['Close']

data = pd.DataFrame()

data['Close'] = closes
data['Rolling Average'] = closes.rolling(window=200).mean()
data['Rolling Minimum'] = closes.rolling(window=200).min()
data['Rolling Maximum'] = closes.rolling(window=200).max()

percent_boundary = 0.6  # because the website said so lol can tinker w this later
data['Lower Boundary'] = (0.5*(data['Rolling Minimum'] + data['Rolling Maximum'])) - (0.5*0.6*(data['Rolling Maximum'] - data['Rolling Minimum']))
data['Upper Boundary'] = (0.5*(data['Rolling Minimum'] + data['Rolling Maximum'])) + (0.5*0.6*(data['Rolling Maximum'] - data['Rolling Minimum']))
data['Lower Boundary Indicator'] = np.where(data['Close'] > data['Rolling Minimum'], 100, 0)
data['Upper Boundary Indicator'] = np.where(data['Close'] < data['Rolling Maximum'], 100, 0)

data['Close Prior 3 Weeks'] = closes.shift(periods=21)
data['3 Week Prior Change'] = data['Close'] - data['Close Prior 3 Weeks']

data['Close After 1 Week'] = closes.shift(periods=-7)
data['1 Week After Change'] = data['Close After 1 Week'] - data['Close']

data = data['2016-12-16':'2023-01-01']

print(data)

                 Close  Rolling Average  Rolling Minimum  Rolling Maximum  \
date                                                                        
2016-12-16  200.451508       187.174848       173.007095       201.685562   
2016-12-19  200.887985       187.304704       173.007095       201.685562   
2016-12-20  201.662949       187.447983       173.861725       201.685562   
2016-12-21  201.101761       187.584183       174.001251       201.685562   
2016-12-22  200.754379       187.717949       175.861038       201.685562   
...                ...              ...              ...              ...   
2022-12-23  378.706665       392.922738       351.033936       450.591370   
2022-12-27  377.213226       392.764673       351.033936       450.591370   
2022-12-28  372.525238       392.598107       351.033936       450.591370   
2022-12-29  379.230835       392.420447       351.033936       450.591370   
2022-12-30  378.231903       392.191807       351.033936       450.591370   

### Build Various SK-Learn Mean Reversion Models

In [175]:
# building and testing models

def build_model(prices, parameters):
    X = prices[parameters].values
    y = prices['1 Week After Change'].values

    lasso_model = sklearn.linear_model.Lasso(alpha=1.0)  # You can specify the regularization strength (alpha)
    lasso_model.fit(X, y)

    return lasso_model

def test_model(model, dates, parameters):
    return model.predict(dates[parameters].values)

In [176]:
# establishing different model parameters:
# model 1: close, rolling average
# model 2: close, rolling average, 3 wk change
# model 3: close, rolling average, rolling min, rolling max
# model 4: close, rolling average, rolling min, rolling max, 3 wk change
# model 5: close, rolling average, lower boundary, upper boundary
# model 6: close, rolling average, lower boundary, upper boundary, 3 wk change
# model 7: close, rolling average, lower boundary indicator, upper boundary indicator
# model 8: close, rolling average, lower boundary indicator, upper boundary indicator, 3 wk change

def get_parameters(version):
    m1params = ['Close','Rolling Average']
    m2params = ['Close','Rolling Average','3 Week Prior Change']
    m3params = ['Close','Rolling Average','Rolling Minimum','Rolling Maximum']
    m4params = ['Close','Rolling Average','Rolling Minimum','Rolling Maximum','3 Week Prior Change']
    m5params = ['Close','Rolling Average','Lower Boundary','Upper Boundary']
    m6params = ['Close','Rolling Average','Lower Boundary','Upper Boundary','3 Week Prior Change']
    m7params = ['Close','Rolling Average','Lower Boundary Indicator','Upper Boundary Indicator']
    m8params = ['Close','Rolling Average','Lower Boundary Indicator','Upper Boundary Indicator','3 Week Prior Change']
    params = [m1params,m2params,m3params,m4params,m5params,m6params,m7params,m8params]
    return params[version]

### Generating Entry and Exit Signals

In [177]:
# entry and exit signals are a pandas series of boolean values indicating when a stock should be bought and sold

def get_signals(pred_returns):
    signals = pd.DataFrame()
    
    signals['Base Entries'] = np.where(pred_returns > 0, True, False)
    signals['Base Exits'] = np.where(pred_returns < 0, True, False)

    max=pred_returns.max()
    min=pred_returns.min()
    pos_avg=np.mean(pred_returns[np.where(pred_returns>0)])
    neg_avg=np.mean(pred_returns[np.where(pred_returns<0)])

    signals['0.25 Entries'] = np.where(pred_returns > 0.25*max, True, False)
    signals['0.25 Exits'] = np.where(pred_returns < 0.25*min, True, False)

    signals['0.5 Entries'] = np.where(pred_returns > 0.5*max, True, False)
    signals['0.5 Exits'] = np.where(pred_returns < 0.5*min, True, False)

    signals['0.25-0.75 Entries'] = np.where((pred_returns > 0.25*max) & (pred_returns < 0.75*max), True, False)
    signals['0.25-0.75 Exits'] = np.where((pred_returns < 0.25*min) & (pred_returns > 0.75*min), True, False)

    signals['0.5-0.8 Entries'] = np.where((pred_returns > 0.5*max) & (pred_returns < 0.8*max), True, False)
    signals['0.5-0.8 Exits'] = np.where((pred_returns < 0.5*min) & (pred_returns > 0.8*min), True, False)

    signals['Above Average Entries'] = np.where(pred_returns > pos_avg, True, False)
    signals['Above Average Exits'] = np.where(pred_returns < neg_avg, True, False)

    signals['Around Average Entries'] = np.where((pred_returns > 0.5*pos_avg) & (pred_returns < 2*pos_avg), True, False)
    signals['Around Average Exits'] = np.where((pred_returns < 0.5*neg_avg) & (pred_returns > 2*neg_avg), True, False)

    return signals

### Backtest SK-Learn Model with VectorBT

In [178]:
figure = data['Close'].vbt.rolling_split(n=12, window_len=475, set_lens=(95,),left_to_right=False,plot=True)
figure.update_layout(width=640,height=360)
figure.show()

print(list(range(0,7)))

[0, 1, 2, 3, 4, 5, 6]


In [179]:
# backtesting code

(in_sample_prices,in_sample_dates), (out_sample_prices,out_sample_dates) = data['Close'].vbt.rolling_split(n=12, window_len=475, set_lens=(95,),left_to_right=False)

signal_names = [('Base Entries','Base Exits'),('0.25 Entries','0.25 Exits'),('0.5 Entries','0.5 Exits'),('0.25-0.75 Entries','0.25-0.75 Exits'),('0.5-0.8 Entries','0.5-0.8 Exits'),('Above Average Entries','Above Average Exits'),('Around Average Entries','Around Average Exits')]

models_idx = list(range(0,8))
signals_idx = list(range(0,7))
returns_idx = pd.MultiIndex.from_tuples([(m,s) for m in models_idx for s in signals_idx], names=['model_version','signal_level'])
returns = pd.DataFrame(columns=returns_idx)
sharpe = pd.DataFrame(columns=returns_idx)

for m in range(0,8):
    for s in range(0,7):
        split_idx_list = list(range(0,12))
        split_idx = pd.MultiIndex.from_tuples([(x,) for x in split_idx_list], names=['split_idx'])
        entries = pd.DataFrame(columns=split_idx)
        exits = pd.DataFrame(columns=split_idx)
        for i in range(0,12):
            model = build_model(data[in_sample_dates[i][0]:in_sample_dates[i][-1]], get_parameters(m))
            pred = test_model(model, data[out_sample_dates[i][0]:out_sample_dates[i][-1]], get_parameters(m))
            signals = get_signals(pred)
            entries.loc[:, i] = signals[signal_names[s][0]]
            exits.loc[:, i] = signals[signal_names[s][1]]
        pf=vbt.Portfolio.from_signals(out_sample_prices, entries, exits, freq='1d', direction='both')
        res=pd.DataFrame({'Total Return': pf.total_return(), 'Sharpe Ratio': pf.sharpe_ratio()})
        returns.loc[:, (m,s)] = pf.total_return()
        sharpe.loc[:, (m,s)] = pf.sharpe_ratio()
        # print(f"Model {m+1} ({signal_names[s][0]}, {signal_names[s][1]}):")
        # print(res)

# print(returns)

returns_sums = (returns.sum()).sort_values(ascending=False)
sharpe_averages = sharpe.mean()

results = pd.DataFrame(index=returns_sums.index)
results['returns sums'] = returns_sums.values
results['sharpe avgs'] = sharpe_averages.reindex(results.index).values
print(results)

                            returns sums  sharpe avgs
model_version signal_level                           
3             2                 1.030936     1.564421
5             2                 1.021257     1.602902
              4                 0.881981     1.403316
              5                 0.874271     1.553203
3             5                 0.798130     1.406726
4             2                 0.741380     1.437707
2             2                 0.737083     1.437193
3             4                 0.736369     1.135540
2             5                 0.731493     1.379807
4             5                 0.725758     1.368459
3             1                 0.688339     1.262836
5             1                 0.681714     1.219833
2             4                 0.632076     1.268026
4             4                 0.630490     1.251032
6             5                 0.625439     0.876962
              2                 0.603127     0.891864
0             2             

### Validation of Results

In [180]:
final_closes = (openbb.stocks.load(symbol="SPY", start_date="2020-01-01", end_date="2023-01-01"))['Close']

final_data = pd.DataFrame()

final_data['Close'] = final_closes
final_data['Rolling Average'] = final_closes.rolling(window=200).mean()
final_data['Rolling Minimum'] = final_closes.rolling(window=200).min()
final_data['Rolling Maximum'] = final_closes.rolling(window=200).max()

percent_boundary = 0.6  # because the website said so lol can tinker w this later
final_data['Lower Boundary'] = (0.5*(final_data['Rolling Minimum'] + final_data['Rolling Maximum'])) - (0.5*0.6*(final_data['Rolling Maximum'] - final_data['Rolling Minimum']))
final_data['Upper Boundary'] = (0.5*(final_data['Rolling Minimum'] + final_data['Rolling Maximum'])) + (0.5*0.6*(final_data['Rolling Maximum'] - final_data['Rolling Minimum']))
final_data['Lower Boundary Indicator'] = np.where(final_data['Close'] > final_data['Rolling Minimum'], 100, 0)
final_data['Upper Boundary Indicator'] = np.where(final_data['Close'] < final_data['Rolling Maximum'], 100, 0)

final_data['Close Prior 3 Weeks'] = final_closes.shift(periods=21)
final_data['3 Week Prior Change'] = final_data['Close'] - final_data['Close Prior 3 Weeks']

final_data['Close After 1 Week'] = final_closes.shift(periods=-7)
final_data['1 Week After Change'] = final_data['Close After 1 Week'] - final_data['Close']

final_data = final_data['2021-06-21':'2022-12-20']

print(final_data.shape)


INFO:openbb_terminal.stocks.stocks_helper:START
INFO:openbb_terminal.stocks.stocks_helper:{"INPUT": {"start_date": "2020-01-01", "interval": "1440", "end_date": "2023-01-01", "prepost": "False", "source": "YahooFinance", "weekly": "False", "monthly": "False", "verbose": "True", "symbol": "SPY", "chart": "False"}, "VIRTUAL_PATH": "stocks.load", "CHART": false}


Loading Daily data for SPY with starting period 2020-01-02.

INFO:openbb_terminal.stocks.stocks_helper:END


(380, 12)


In [181]:
future_closes = (openbb.stocks.load(symbol="SPY", start_date="2021-01-01", end_date="2023-05-18"))['Close']

future_data = pd.DataFrame()

future_data['Close'] = future_closes
future_data['Rolling Average'] = future_closes.rolling(window=200).mean()
future_data['Rolling Minimum'] = future_closes.rolling(window=200).min()
future_data['Rolling Maximum'] = future_closes.rolling(window=200).max()

percent_boundary = 0.6  # because the website said so lol can tinker w this later
future_data['Lower Boundary'] = (0.5*(future_data['Rolling Minimum'] + future_data['Rolling Maximum'])) - (0.5*0.6*(future_data['Rolling Maximum'] - future_data['Rolling Minimum']))
future_data['Upper Boundary'] = (0.5*(future_data['Rolling Minimum'] + future_data['Rolling Maximum'])) + (0.5*0.6*(future_data['Rolling Maximum'] - future_data['Rolling Minimum']))
future_data['Lower Boundary Indicator'] = np.where(future_data['Close'] > future_data['Rolling Minimum'], 100, 0)
future_data['Upper Boundary Indicator'] = np.where(future_data['Close'] < future_data['Rolling Maximum'], 100, 0)

future_data['Close Prior 3 Weeks'] = future_closes.shift(periods=21)
future_data['3 Week Prior Change'] = future_data['Close'] - future_data['Close Prior 3 Weeks']

future_data = future_data["2023-01-01":"2023-05-18"]

print(future_data.shape)

INFO:openbb_terminal.stocks.stocks_helper:START
INFO:openbb_terminal.stocks.stocks_helper:{"INPUT": {"start_date": "2021-01-01", "interval": "1440", "end_date": "2023-05-18", "prepost": "False", "source": "YahooFinance", "weekly": "False", "monthly": "False", "verbose": "True", "symbol": "SPY", "chart": "False"}, "VIRTUAL_PATH": "stocks.load", "CHART": false}


Loading Daily data for SPY with starting period 2021-01-04.

INFO:openbb_terminal.stocks.stocks_helper:END


(95, 10)


In [182]:
models_idx = list(range(0,8))
signals_idx = list(range(0,7))
future_returns_idx = pd.MultiIndex.from_tuples([(m,s) for m in models_idx for s in signals_idx], names=['model_version','signal_level'])
future_returns = pd.DataFrame(columns=future_returns_idx)

for m in range(0,8):
    for s in range(0,7):
        model = build_model(final_data, get_parameters(m))
        pred = test_model(model, future_data, get_parameters(m))
        signals = get_signals(pred)
        entries = signals[signal_names[s][0]]
        exits = signals[signal_names[s][1]]
        pf=vbt.Portfolio.from_signals(future_closes["2023-01-01":"2023-05-18"].values, entries, exits, freq='1d', direction='both')
        future_returns.loc[:, (m,s)] = [pf.total_return()]

print(future_returns)
future_returns_sums = future_returns.sum()
print(future_returns_sums)

results['future returns'] = future_returns_sums.reindex(results.index).values
print(results)


model_version         0                                                    \
signal_level          0         1         2         3         4         5   
0              0.105063  0.105063  0.105063  0.063992  0.063992  0.105063   

model_version                   1                      ...         6  \
signal_level          6         0         1         2  ...         4   
0              0.105063  0.105063  0.105063  0.105063  ...  0.063992   

model_version                             7                               \
signal_level          5         6         0         1         2        3   
0              0.105063  0.105063  0.105063  0.105063  0.105063  0.08439   

model_version                               
signal_level         4         5         6  
0              0.08439  0.105063  0.105063  

[1 rows x 56 columns]
model_version  signal_level
0              0               0.105063
               1               0.105063
               2               0.105063
               3

### Conclusion:

While I was able to combine most of the libraries to create an interesting program, the overall effectiveness of the program was extremely low.